In [ ]:
#uncomment the library needed to install

%pip install sentence-transformers 
#%pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
#%pip install xlsxwriter
#%pip install pandas
#%pip install numpy

In [2]:
# these are the libraries that we need
import pandas as pd
import os
import numpy as np
import torch
from sentence_transformers import SentenceTransformer #install with pip install -U sentence-transformers
import pickle
import warnings
warnings.filterwarnings('ignore')

/home/codespace/.python/current/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [ ]:
# load our dataset of SQL log
df = pd.read_excel('/kaggle/input/sql-examples/SQL_random.xlsx', engine='openpyxl')


In [6]:
# carichiamo il file training.txt come df proviamo con un log non SQL
df = pd.read_csv('training.txt', sep='\t', header=None)
df

,0
0,Apr 15 2013 09:36:50: %ASA-4-106023: Deny tcp ...
1,Apr 15 2013 09:36:50: %ASA-4-106023: Deny tcp ...
2,Apr 15 2014 09:34:34 EDT: %ASA-session-5-10610...
3,Apr 24 2013 16:00:28 INT-FW01 : %ASA-6-106100:...
4,Apr 24 2013 16:00:27 INT-FW01 : %ASA-6-106100:...
5,Apr 29 2013 12:59:50: %ASA-6-305011: Built dyn...
6,Apr 29 2013 12:59:50: %ASA-6-302013: Built out...
7,Apr 29 2013 12:59:50: %ASA-6-305011: Built dyn...
8,Apr 29 2013 12:59:50: %ASA-6-302015: Built out...
9,Apr 29 2013 12:59:50: %ASA-6-305011: Built dyn...


In [7]:
#rinominimo le colonne, la prima si chiama 'String'
df.columns = ['String']
df

,String
0,Apr 15 2013 09:36:50: %ASA-4-106023: Deny tcp ...
1,Apr 15 2013 09:36:50: %ASA-4-106023: Deny tcp ...
2,Apr 15 2014 09:34:34 EDT: %ASA-session-5-10610...
3,Apr 24 2013 16:00:28 INT-FW01 : %ASA-6-106100:...
4,Apr 24 2013 16:00:27 INT-FW01 : %ASA-6-106100:...
5,Apr 29 2013 12:59:50: %ASA-6-305011: Built dyn...
6,Apr 29 2013 12:59:50: %ASA-6-302013: Built out...
7,Apr 29 2013 12:59:50: %ASA-6-305011: Built dyn...
8,Apr 29 2013 12:59:50: %ASA-6-302015: Built out...
9,Apr 29 2013 12:59:50: %ASA-6-305011: Built dyn...


In [8]:
#we check if GPU is available, for big dataset is necessary for embedding
if torch.cuda.is_available():
    device_count = torch.cuda.device_count()
    print(f"Number of GPUs available: {device_count}")
    for i in range(device_count):
        device = torch.device(f"cuda:{i}")
        print(f"Device {i}: {torch.cuda.get_device_name(device)}")
else:
    device = torch.device("cpu")
    print("CUDA is not available")

CUDA is not available


In [9]:
# we load a little, fast but accurate model, more info on www.sbert.net
modelST = 'all-MiniLM-L6-v2'
model = SentenceTransformer(modelST, device=device) 

# the function to create the embedding
def embeddings(text):
    return model.encode(text, device = device, batch_size=64, normalize_embeddings=True) #normalize_embeddings – If set to true, returned vectors will have length 1. In that case, the faster dot-product (util.dot_score) instead of cosine similarity can be used.

/home/codespace/.python/current/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [10]:
df['String_Embedded'] = df['String'].apply(embeddings)

In [ ]:
# we can store the embedding to disk so we don't have to recompute every time
with open('embeddings_test.pkl', 'wb') as fOut:
    pickle.dump(df['String_Embedded'], fOut, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#and if needed we can load it
#Load sentences & embeddings from disc
with open('embeddings_test.pkl', "rb") as fIn:
    stored_data = pickle.load(fIn)
    df['String_Embedded'] = stored_data['embeddings_test']

In [11]:
#let'see the dataframe now
df

,String,String_Embedded
0,Apr 15 2013 09:36:50: %ASA-4-106023: Deny tcp ...,"[-0.076410495, 0.10827733, 0.0008779595, 0.049..."
1,Apr 15 2013 09:36:50: %ASA-4-106023: Deny tcp ...,"[-0.07741436, 0.10693011, -0.008771725, 0.0573..."
2,Apr 15 2014 09:34:34 EDT: %ASA-session-5-10610...,"[-0.0853249, 0.12564649, 0.004367104, 0.048236..."
3,Apr 24 2013 16:00:28 INT-FW01 : %ASA-6-106100:...,"[-0.05257576, 0.15259308, 0.023978252, 0.05431..."
4,Apr 24 2013 16:00:27 INT-FW01 : %ASA-6-106100:...,"[-0.05345528, 0.12695977, 0.02379341, 0.030163..."
5,Apr 29 2013 12:59:50: %ASA-6-305011: Built dyn...,"[-0.08928572, 0.10466742, -0.005363382, 0.0425..."
6,Apr 29 2013 12:59:50: %ASA-6-302013: Built out...,"[-0.07798385, 0.10975029, -0.008579611, 0.0489..."
7,Apr 29 2013 12:59:50: %ASA-6-305011: Built dyn...,"[-0.044473458, 0.027816104, 0.004526236, -0.01..."
8,Apr 29 2013 12:59:50: %ASA-6-302015: Built out...,"[-0.04556694, 0.028472586, 0.009824611, 0.0035..."
9,Apr 29 2013 12:59:50: %ASA-6-305011: Built dyn...,"[-0.045155577, 0.042557497, -0.0074583692, -0...."


In [13]:
# now we calculate the average value, we use a library that use multithread, again, is usefull if our dataset is big
import concurrent.futures

def calculate_mean_embedding(df):
    return np.mean(df['String_Embedded'])

with concurrent.futures.ThreadPoolExecutor() as executor:
    future = executor.submit(calculate_mean_embedding, df)
    embedding_avg = future.result()

print('the average value of the embedded strings is:', embedding_avg)

the average value of the embedded strings is: [-5.62616438e-02  7.40311220e-02  4.40562423e-03  3.76007147e-02
  3.11746038e-02 -3.96879278e-02  5.01967268e-03 -3.13383974e-02
  4.70643537e-03 -8.89511965e-03 -1.93249397e-02 -1.19363731e-02
 -4.88590971e-02  1.33799920e-02  6.44010399e-03  6.68800101e-02
  2.21524742e-02 -1.02495559e-01 -2.94999406e-02 -1.24668907e-02
 -2.02599429e-02  6.97711408e-02 -1.93576347e-02 -3.68245915e-02
  3.31955234e-04 -1.58709604e-02 -1.50959694e-03  3.55383218e-03
  5.56996241e-02 -4.87425923e-02 -3.29722241e-02  1.09530904e-03
 -4.20833603e-02  3.86722852e-03  1.89473499e-02 -3.19841616e-02
  7.28775188e-02 -1.24340076e-02  3.64442579e-02  4.83359024e-02
  1.08917102e-01  3.79512608e-02  2.81722769e-02  2.30623018e-02
 -1.75322313e-02  3.36304074e-03 -2.65273061e-02  6.89318345e-04
  7.09442869e-02  2.22703852e-02 -5.18690608e-02  7.61652663e-02
 -3.67167071e-02  1.71687864e-02 -1.89841725e-04 -9.12228785e-03
  1.17629627e-02  1.52622620e-02 -6.43845135

In [14]:
# now that we have the 'average meaning encoded' we check the similarity of all the others queries

# Calculate the anomaly score for each SQL string related to embedding_avg
print('Computing anomalies...')
scores = np.dot(df['String_Embedded'].tolist(), embedding_avg)

# to make easier to read, we convert the value to integers from 0 to 10
scores_abs = [int(score*10) for score in scores]

# we add the value to the dataset
df['Dataset_Anomaly'] = scores_abs

#we print some brief report
print('Number of Dataset_Anomaly 0: ', len(df[df['Dataset_Anomaly'] == 0]))
print('Number of Dataset_Anomaly 1: ', len(df[df['Dataset_Anomaly'] == 1]))
print('Number of Dataset_Anomaly 2: ', len(df[df['Dataset_Anomaly'] == 2]))
print('Number of Dataset_Anomaly 3: ', len(df[df['Dataset_Anomaly'] == 3]))

Computing anomalies...
Number of Dataset_Anomaly 0:  0
Number of Dataset_Anomaly 1:  0
Number of Dataset_Anomaly 2:  0
Number of Dataset_Anomaly 3:  0


In [ ]:
#so we have now for every line a score of anomaly, the most interesting are low value
df

In [15]:
#we can save it as a new file, then analyze with Excel
df.to_excel('results_training.xlsx', index=False, engine='openpyxl')